# FINAL PROJECT DATN - TRỢ LÝ ẢO CHUYÊN VỀ QUY HOẠCH ĐÔ THỊ VÀ GIAO THÔNG
- Author:
  - Đặng Kim Thành - 21110298
  - Bùi Quốc Khang - 21110202
## Phần 3 - TrafficChat ViVi - Retrieval-Augmented Generation Advancde

### I. Xử lý dữ liệu

#### Tải môi trường thư viện và kiểm tra GPU

In [3]:
import unidecode
import re
import pdfplumber
import os
import pickle
import json
from langchain.schema import Document
import numpy as np

In [4]:
import torch
print(torch.cuda.is_available())

True


#### Xử lý tên file của dữ liệu (đề phòng trường hợp lỗi khi tải dữ liệu)

In [7]:
pdf_path = "./Data_Traffic/PDF"  # Đường dẫn tới thư mục chứa các tệp cần đổi tên

In [8]:
# Duyệt qua tất cả các tệp trong thư mục
for filename in os.listdir(pdf_path):
    # Chuyển đổi tên tệp sang không dấu
    new_filename = unidecode.unidecode(filename)
    # Thay thế dấu cách bằng dấu gạch dưới
    
    new_filename = new_filename.replace(' ', '_')
    # Loại bỏ các ký tự đặc biệt, chỉ giữ lại chữ cái, số và dấu gạch dưới
    new_filename = re.sub(r'[^A-Za-z0-9_.]', '', new_filename)

    # Đổi tên tệp
    os.rename(os.path.join(pdf_path, filename), os.path.join(pdf_path, new_filename))
    print(f"Đã đổi tên tệp: {filename} -> {new_filename}")

Đã đổi tên tệp: Huong_dan_cho_cac_duong_pho.pdf -> Huong_dan_cho_cac_duong_pho.pdf
Đã đổi tên tệp: Ly_thuyet_luu_luong_giao_thong.pdf -> Ly_thuyet_luu_luong_giao_thong.pdf
Đã đổi tên tệp: Ly_thuyet_thiet_ke_do_thi_duong_xa.pdf -> Ly_thuyet_thiet_ke_do_thi_duong_xa.pdf
Đã đổi tên tệp: Nhan_thuc_tinh_huong_phan_tan_trong_giao_thong_duong_bo_Ly_thuyet_do_luong_va_ung_dung_vao_thiet_ke_giao_lo.pdf -> Nhan_thuc_tinh_huong_phan_tan_trong_giao_thong_duong_bo_Ly_thuyet_do_luong_va_ung_dung_vao_thiet_ke_giao_lo.pdf
Đã đổi tên tệp: Quan_ly_quy_hoach_do_thi_thong_qua_he_thong_dia_ly_GIS.pdf -> Quan_ly_quy_hoach_do_thi_thong_qua_he_thong_dia_ly_GIS.pdf
Đã đổi tên tệp: Quy_hoach_to_chuc_va_dieu_khien_giao_thong_do_thi.pdf -> Quy_hoach_to_chuc_va_dieu_khien_giao_thong_do_thi.pdf
Đã đổi tên tệp: Thuc_hanh_tot_de_dam_bao_an_toan_giao_thong_do_thi.pdf -> Thuc_hanh_tot_de_dam_bao_an_toan_giao_thong_do_thi.pdf


#### Xử lý dữ liệu đầu vào PDF

In [9]:
# ✅ Normalize filenames for consistency
def normalize_filename(filename):
    filename = unidecode.unidecode(filename)
    filename = filename.replace(" ", "_")
    return filename

# ✅ Remove page numbers from text
def remove_page_numbers(text):
    # Remove lines that only contain numbers (likely page numbers)
    lines = text.split('\n')
    cleaned_lines = [line for line in lines if not re.match(r'^\s*\d+\s*$', line.strip())]
    return "\n".join(cleaned_lines)

# ✅ Clean and merge fragmented lines
def clean_and_merge_lines(lines):
    merged_lines = []
    buffer = ""
    for line in lines:
        stripped_line = line.strip()
        if buffer:
            if stripped_line and not stripped_line[0].isupper():
                buffer += " " + stripped_line  # Merge fragmented lines
            else:
                merged_lines.append(buffer)
                buffer = stripped_line
        else:
            buffer = stripped_line
    if buffer:
        merged_lines.append(buffer)
    return merged_lines

# ✅ Read a PDF file using pdfplumber and apply all processing
def read_pdf_with_plumber(file_path):
    content = ""
    try:
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    text = remove_page_numbers(text)
                    content += text + "\n"
    except Exception as e:
        print(f"❌ Error reading {file_path}: {e}")
    # Xử lý ghép dòng
    lines = content.split("\n")
    merged_lines = clean_and_merge_lines(lines)
    return "\n".join(merged_lines)

# ✅ Read all PDF files in a folder and return a dictionary with normalized filename and cleaned content
def read_all_pdfs_with_plumber(folder_path):
    all_contents = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            file_path = os.path.join(folder_path, filename)
            content = read_pdf_with_plumber(file_path)
            normalized_name = normalize_filename(os.path.splitext(filename)[0])
            all_contents[normalized_name] = content
    return all_contents


In [10]:
pdf_books = read_all_pdfs_with_plumber(pdf_path)

# 🖨️ In thử một nội dung đã xử lý
for file, text in pdf_books.items():
    print(f"====== {file} ======")
    print(text[:1000])  # In 1000 ký tự đầu
    break

====== Huong_dan_cho_cac_duong_pho ======
Manual for Streets
Manual for Streets
Published by Thomas Telford Publishing, Thomas Telford Ltd, 1 Heron Quay, London E14 4JD. www.thomastelford.com
Distributors for Thomas Telford books are
USA: ASCE Press, 1801 Alexander Bell Drive, Reston, VA 20191-4400, USA
Japan: Maruzen Co. Ltd, Book Department, 3–10 Nihonbashi 2-chome, Chuo-ku, Tokyo 103
Australia: DA Books and Journals, 648 Whitehorse Road, Mitcham 3132, Victoria
First published 2007
Published for the Department for Transport under licence from the Controller of Her Majesty’s Stationery Office © Queen’s Printer and Controller of HMSO, 2007
Copyright in the typographical arrangement and design rests with the Queen’s Printer and Controller of HMSO.
This publication (excluding logos) may be reproduced free of charge in any format or medium for non-commercial research, private study or for circulation within an organisation. This is subject to it being reproduced accurately and not used in

In [11]:
# Biến đổi nội dung PDF thành các Document của LangChain
def extract_title_from_content(content):
    for line in content.split("\n"):
        if len(line.strip()) > 10 and line.strip().istitle():
            return line.strip()
    return None

def extract_main_topic(content):
    topic_patterns = [
        r"Chủ đề[:\-]\s*(.+)",
        r"Topic[:\-]\s*(.+)",
        r"Category[:\-]\s*(.+)",
        r"Chương\s+\d+[:\-]?\s*(.+)",
        r"Phần\s+\d+[:\-]?\s*(.+)"
    ]
    for line in content.split('\n')[:20]:  # Chỉ quét 20 dòng đầu
        for pattern in topic_patterns:
            match = re.search(pattern, line, re.IGNORECASE)
            if match:
                return match.group(1).strip()
    return "Unknown"

def save_pdf_books_as_json(pdf_books, save_path):
    saved_data = []
    for filename, content in pdf_books.items():
        title = extract_title_from_content(content) or filename.replace("_", " ")
        category = extract_main_topic(content)
        saved_data.append({
            "filename": filename,
            "content": content.strip(),
            "title": title,
            "category": category
        })
    
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(saved_data, f, ensure_ascii=False, indent=2)

    print(f"✅ Đã lưu {len(saved_data)} mục vào {save_path}")

# Lưu data pdf lại
save_pdf_books_as_json(pdf_books, os.path.join(pdf_path, "processed_pdf_documents.json"))

✅ Đã lưu 7 mục vào ./Data_Traffic/PDF\processed_pdf_documents.json


In [13]:
def load_documents_from_json(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    documents = []
    for item in data:
        doc = Document(
            page_content=item["content"],
            metadata={
                "source": item["filename"],
                "title": f"Tiêu đề: {item['title']}",
                "category": item["category"]
            }
        )
        documents.append(doc)
    return documents

# ✅ Gọi hàm để tạo danh sách tài liệu LangChain
pdf_documents = load_documents_from_json(os.path.join(pdf_path, "processed_pdf_documents.json"))
print(f"Số lượng Document đã load: {len(pdf_documents)}")

Số lượng Document đã load: 7


#### Xử lý dữ liệu đầu vào JSON

In [30]:
# Đọc tệp JSON
def load_json_data(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        return json.load(f)

def ensure_string(value):
    if isinstance(value, list):
        # Nếu là danh sách dict, chuyển dict thành chuỗi
        return "\n".join(
            [json.dumps(v, ensure_ascii=False) if isinstance(v, dict) else str(v) for v in value]
        )
    elif isinstance(value, dict):
        return json.dumps(value, ensure_ascii=False)
    elif isinstance(value, str):
        return value
    return str(value)  # fallback

# Chuyển dữ liệu từ JSON thành định dạng Document của LangChain
def convert_json_to_documents(json_data):
    documents = []

    # Duyệt qua các disease trong dữ liệu JSON
    for main_topic, items in json_data.items():
        for item in items:
            title = item.get("title", "")
            description = ensure_string(item.get("description", ""))
            causes = ensure_string(item.get("causes", ""))
            mechanism = ensure_string(item.get("mechanism", ""))
            meaning = ensure_string(item.get("meaning", ""))

            # Tạo content và metadata cho mỗi Document
            content = "Mô tả chủ đề: " + description + "\nNguyên nhân: " + causes + "\nPhương pháp thực hiện: " + mechanism + "\nÝ nghĩa của hành vi: " + meaning
            metadata = {
                "title": f"Tiêu đề: {title}",
                "category": main_topic
            }

            # Tạo Document
            document = Document(page_content=content, metadata=metadata)
            documents.append(document)

    return documents

# Hàm để load tất cả các tệp JSON trong thư mục
def load_all_json_files_in_directory(directory_path):
    documents = []
    # Duyệt qua tất cả các tệp trong thư mục
    for filename in os.listdir(directory_path):
        # Kiểm tra nếu tệp có phần mở rộng là .json
        if filename.endswith('.json'):
            json_file_path = os.path.join(directory_path, filename)
            # Đọc và chuyển tệp JSON thành documents
            json_data = load_json_data(json_file_path)
            documents.extend(convert_json_to_documents(json_data))
            print(f"Loaded: {filename}")
    return documents

In [31]:
# Đường dẫn tới thư mục chứa các tệp JSON
json_directory = "./Data_Traffic/JSON"  
# Load tất cả các tệp JSON trong thư mục
json_documents = load_all_json_files_in_directory(json_directory)

Loaded: quy_hoach_traffic.json


In [32]:
print(f"✅ Đã tạo {len(json_documents)} Documents")
print(json_documents[3].page_content)
print(json_documents[3].metadata)

✅ Đã tạo 562 Documents
Mô tả chủ đề: Cung cấp vị trí trạm, tuyến xe, khả năng tiếp cận cho người khuyết tật, thời gian đến dự kiến.
Nguyên nhân: Người dùng cần thông tin đa chiều để lựa chọn tuyến phù hợp, đặc biệt người khuyết tật.
Phương pháp thực hiện: Tích hợp dữ liệu GTFS + GTFS‑Realtime, cùng OSM để tính khoảng cách và đánh giá truy cập.
Ý nghĩa của hành vi: Nâng cao khả năng lập kế hoạch cá nhân, hỗ trợ người yếu thế, khuyến khích sử dụng giao thông công cộng.
{'title': 'Tiêu đề: Định vị trạm xe buýt & loại hình giao thông công cộng', 'category': 'Phân bố luồng giao thông và Quy hoạch đô thị'}


#### Xử lý dữ liệu đầu vào TXT

In [12]:
def parse_txt_to_documents(raw_text):
    documents = []
    current_category = None
    lines = raw_text.strip().split("\n")

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Phát hiện đề mục
        if line.startswith("===") and line.endswith("==="):
            current_category = line.strip("= ").strip()
            continue

        # Tìm tên vị trí (tên đường + camera)
        match = re.search(r"camera.*?tại (Xa lộ Hà Nội.*?Đường [^,–-]*)", line)
        if not match:
            match = re.search(r"tại (Xa lộ Hà Nội.*?Đường [^,–-]*)", line)
        title_location = match.group(1).strip() if match else "Không rõ vị trí"

        # Ghép thành title hoàn chỉnh
        full_title = f"{current_category} tại vị trí {title_location}"

        # Tạo document
        doc = Document(
            page_content=line,
            metadata={
                "title": f"Tiêu đề: {full_title}",
                "category": current_category or "Không rõ chủ đề"
            }
        )
        documents.append(doc)

    return documents


# ✅ Đọc toàn bộ file .txt trong thư mục
def load_documents_from_directory(directory):
    all_documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r", encoding="utf-8") as f:
                raw_text = f.read()
            docs = parse_txt_to_documents(raw_text)  # <-- raw_text, không phải filepath
            all_documents.extend(docs)
    return all_documents

In [13]:
# Đường dẫn tới thư mục chứa các tệp TXT
txt_directory = "./Data_Traffic/TXT"  
# Load tất cả các tệp TXT trong thư mục
txt_documents = load_documents_from_directory(txt_directory)
len(txt_documents)

200

In [14]:
print(txt_documents[3].page_content)
print(txt_documents[3].metadata)

Vào ngày 10 tháng 6 năm 2025, tại camera được đặt tại Xa lộ Hà Nội – Đường D1, thuộc khu vực Phường Thảo Điền của Quận 2, TP.HCM), trong khoảng thời gian 19:00–19:30, hệ thống ghi nhận 40 phương tiện loại "xe ô tô" di chuyển từ hướng "Bắc".
{'title': 'Tiêu đề: Thống kê lượng xe cộ tại vị trí Xa lộ Hà Nội – Đường D1', 'category': 'Thống kê lượng xe cộ'}


In [ ]:
# Thư mục chứa file txt
folder_path = "./Data_Traffic/prompt_txt"
filenames = ["traffic-data-1.txt", "traffic-data-2.txt"]

news_documents = []

for file in filenames:
    full_path = os.path.join(folder_path, file)

    # Đọc nội dung file
    with open(full_path, "r", encoding="utf-8") as f:
        raw_data = json.load(f)  # Danh sách các đoạn hội thoại (list of list of dict)

    # Xử lý từng đoạn hội thoại
    for convo in raw_data:
        human = next((m for m in convo if m.get("from") == "human"), None)
        gpt = next((m for m in convo if m.get("from") == "gpt"), None)

        if human and gpt:
            human_text = human["value"]
            gpt_text = gpt["value"]

            # Tách category và title
            if "Tiêu đề:" in human_text:
                before_title, title = human_text.split("Tiêu đề:", 1)
                category = "Thông tin về tình trạng giao thông hằng ngày"
            else:
                title = human_text
                category = "Thông tin về tình trạng giao thông hằng ngày"

            doc = Document(
                page_content=gpt_text.strip(),
                metadata={
                    "title": title.strip(),
                    "category": category.strip()
                }
            )
            news_documents.append(doc)

print(f"✅ Đã tạo {len(news_documents)} Documents")
print(news_documents[3].page_content)
print(news_documents[3].metadata)

✅ Đã tạo 24626 Documents
Nội dung cho toàn bộ sự việc đó là: Nội dung: Liên quan vụ ô tô lùa loạt xe máy trên vỉa hè ở Bình Dương khiến 1 người tử vong, nhiều người bị thương, ngày 16/10, Ban An toàn giao thông tỉnh Bình Dương có báo cáo về vụ việc.
Theo báo cáo của Ban An toàn giao thông tỉnh Bình Dương, nguyên nhân vụ tai nạn giao thông được xác định do tài xế ô tô vi phạm nồng độ cồn, không chú ý quan sát, không đi đúng phần đường quy định.
Hiện trường vụ tai nạn.
Trước đó, chiều 15/10, trong lúc nhiều phụ huynh đỗ xe máy trên vỉa hè đường Huỳnh Văn Lũy (phường Hòa Phú, TP Thủ Dầu Một) để chờ đón học sinh, xe ô tô con BKS: 61K-077.41 chạy với tốc độ cao bất ngờ lao lên vỉa hè, húc văng nhiều phụ huynh.
Cú tông mạnh khiến nhiều người bị thương, trong đó ông Trần Văn L (SN 1963) tử vong tại chỗ.
Tại hiện trường, 5 xe máy nằm la liệt trên vỉa hè, ô tô biến dạng phần đầu.
Sau khi gây tai nạn, tài xế đã rời khỏi hiện trường và đến Công an phường Hòa Phú trình diện. Tài xế được xác định l

#### Chuẩn hóa dữ liệu thành dạng Document của LangChain

In [29]:
# ✅ Gộp cả hai danh sách Document
documents = json_documents + pdf_documents
print(len(documents))
# ✅ In kiểm tra document thứ 6 (nếu đủ phần tử)
if len(documents) >= 7:
    print("🔍 Page content preview:")
    print(documents[9].page_content[:1000])  # In 1000 ký tự đầu
    print("\n📌 Metadata:")
    print(documents[9].metadata)
else:
    print("⚠️ Không đủ 7 tài liệu để kiểm tra document[6]")

569
🔍 Page content preview:
Mô tả chủ đề: Trình bày cách vận dụng lý thuyết dòng xe trong điều phối giao thông đô thị, tối ưu đèn tín hiệu và thiết kế đường.
Nguyên nhân: Giao thông đô thị phức tạp đòi hỏi các giải pháp kỹ thuật để giảm thiểu tắc nghẽn.
Phương pháp thực hiện: Tích hợp mô hình giao thông vào phần mềm mô phỏng (VISSIM, SUMO), kết hợp dữ liệu cảm biến và điều khiển thời gian thực.
Ý nghĩa của hành vi: Nâng cao hiệu quả quản lý hạ tầng, cải thiện trải nghiệm giao thông và giảm phát thải.

📌 Metadata:
{'title': 'Tiêu đề: Ứng dụng lý thuyết dòng xe trong thực tế', 'category': 'Phân bố luồng giao thông và Quy hoạch đô thị'}


### II. Vector hóa và lưu dữ liệu

#### Semantic Chunking and Data split

In [26]:
from transformers import AutoTokenizer

# Load the tokenizer for the BAAI/bge-m3 model
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-v2-m3")

c:\Users\ADMIN\OneDrive\Documents\Btap_Code\VisualStudioCode\Python\TLCN_HealthChatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings

# Khởi tạo HuggingFaceEmbeddings với model BAAI/bge-base-en-v1.5
model_name = "BAAI/bge-base-en-v1.5"
# model_kwargs = {'device': 'cpu'}
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False} # Chuẩn hóa vector = True sẽ limit length vector = 1
hf_embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )

text_splitter = SemanticChunker(hf_embeddings,
                                breakpoint_threshold_type="percentile",
                                breakpoint_threshold_amount=85)

In [34]:
# Split the documents and count the number of chunks in each document, as well as the number of tokens in each chunk
count = 0
total = 0
list_of_chunks = []
for idx,doc in enumerate(documents):
  chunks = text_splitter.create_documents([doc.page_content])
  print(f'Number of chunks: {len(chunks)} - Tokens of each chunk',end=' ')
  for chunk in chunks:
      text = chunk.page_content
      tokens = tokenizer.tokenize(text)
      num_tokens = len(tokens)
      if num_tokens > 1:
        total = total + 1
        # Use the parent document index as metadata to retrieve the parent document from the child chunk.
        chunk.metadata['parent'] = idx
        list_of_chunks.append(chunk)
      if num_tokens > 512:
        count = count + 1
      print(num_tokens, end =' ')
  print()
print('Toltal chunks: ',total)
print('Number of chunks which is larger than 512 tokens: ',count)

Number of chunks: 2 - Tokens of each chunk 58 58 
Number of chunks: 2 - Tokens of each chunk 51 63 
Number of chunks: 2 - Tokens of each chunk 51 57 
Number of chunks: 2 - Tokens of each chunk 84 32 
Number of chunks: 2 - Tokens of each chunk 44 57 
Number of chunks: 2 - Tokens of each chunk 111 28 
Number of chunks: 2 - Tokens of each chunk 49 62 
Number of chunks: 2 - Tokens of each chunk 57 65 
Number of chunks: 2 - Tokens of each chunk 94 29 
Number of chunks: 2 - Tokens of each chunk 57 64 
Number of chunks: 2 - Tokens of each chunk 58 53 
Number of chunks: 2 - Tokens of each chunk 55 47 
Number of chunks: 2 - Tokens of each chunk 61 48 
Number of chunks: 2 - Tokens of each chunk 52 47 
Number of chunks: 2 - Tokens of each chunk 44 54 
Number of chunks: 2 - Tokens of each chunk 72 26 
Number of chunks: 2 - Tokens of each chunk 26 70 
Number of chunks: 2 - Tokens of each chunk 49 47 
Number of chunks: 2 - Tokens of each chunk 43 51 
Number of chunks: 2 - Tokens of each chunk 59 50 

In [35]:
# Chunk example
list_of_chunks[-1]

Document(metadata={'parent': 568}, page_content='and M. Williams (2020), Managing the risk of work-related traffic injuries: Ensuring decent work conditions for those who drive for work and protecting other road users, United Nations Road Safety\nCollaboration publishing, https://app.mapfre.com/documentacion/publico/es/catalogo_imagenes/imagen.do?path=1104746&pos icion=1&registrardownload=1 (accessed 07 May 2020). Prefeitura de Fortaleza (2020), Relatório Preliminar de Vítimas Fatais no Trânsito - Fortaleza 2019 (Preliminary Report of Fatal Traffic Victims - Fortaleza 2019), https://www.unifor.br/documents/929808/930330/Relatorio-Vitimas-Fatais-Fortaleza-2019.pdf (accessed 07 May 2020). Prefeitura de Fortaleza (2018), Página web da Prefeitura de Fortaleza (Municipality of Fortaleza website), https://www.fortaleza.ce.gov.br/ (accessed 30 July 2020). Prefeitura de Fortaleza (2017), Página web da Prefeitura de Fortaleza (Municipality of Fortaleza website), https://www.fortaleza.ce.gov.br/

#### Lưu chuck for bm25 (Chỉ chạy khi chưa lưu dữ liệu lại)

In [36]:
# Lưu dữ liệu Documents ở lần chạy đầu tiên
with open("./vectorstore/db_document/documents.pkl", "wb") as f:
    pickle.dump(list_of_chunks, f)

print("All documents saved successfully!")

All documents saved successfully!


#### Load chuck for bm25 (Chạy khi đã lưu dữ liệu)

In [37]:
# Tải lại tất cả các Document từ tệp pickle
with open("./vectorstore/db_document/documents.pkl", "rb") as f:
    list_of_chunks = pickle.load(f)

print("All documents loaded successfully!")
print(list_of_chunks)

All documents loaded successfully!
[Document(metadata={'parent': 0}, page_content='Mô tả chủ đề: Chatbot cung cấp bản đồ quy hoạch chi tiết, mô tả các khu chức năng (nhà ở, thương mại, xanh...). Nguyên nhân: Người dân, doanh nghiệp cần hiểu rõ các quy hoạch xây dựng để ra quyết định đầu tư và sinh sống.'), Document(metadata={'parent': 0}, page_content='Phương pháp thực hiện: Kết nối dịch vụ GIS, dữ liệu quy hoạch công khai qua API, trả lời theo vị trí hoặc địa chỉ người dùng. Ý nghĩa của hành vi: Tăng tính minh bạch, giúp người dân dễ nắm thông tin và tham gia phản hồi quy hoạch.'), Document(metadata={'parent': 1}, page_content='Mô tả chủ đề: Cung cấp thông tin thời gian, giai đoạn của các dự án như cầu, metro, nâng cấp đường chính. Nguyên nhân: Người dân quan tâm việc đổ lỗi kẹt xe, cấm đường mà không rõ nguyên nhân.'), Document(metadata={'parent': 1}, page_content='Phương pháp thực hiện: Tích hợp nguồn dữ liệu từ cổng thông tin điện tử của Sở và chủ đầu tư, chatbot phản hồi theo gói 

#### Thiết lập id cho các chunks

In [38]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(list_of_chunks))]

In [39]:
uuids

['d728b2ff-49d9-4035-a4e8-b788ffe707d6',
 'd2cdefd6-1adc-42c3-a366-9a908e3f2934',
 'e93aa4b1-f02a-4e46-9ba3-847d64d92d2f',
 'b31610d0-2749-47e8-b07a-24f3b58d4344',
 '1530f025-1610-47ef-9807-54ed5887736e',
 '9fb20865-fc8c-434b-889e-5a493f9b0f25',
 '618e2321-8405-44b8-8fc4-3242d0a046bd',
 '3701ae99-8c10-4913-aefc-ad91a660d2b6',
 '28da49cc-d318-405d-8203-1b922bd4dad4',
 '285c5355-24b1-4730-b605-1a5374f0cf59',
 '8be69d7f-cbf0-4cec-af66-90fa577eb1d9',
 '6a8a4a90-1830-451d-8755-99034f730d9a',
 'ee178c8b-0a33-4784-876b-80bbabeab522',
 '0ed09af2-c5ca-47c2-abe6-1769412a9c76',
 '370e3373-448c-476a-acea-eb0791fb2b4f',
 '9bc2e69f-5ccb-4156-855f-fdec568bc8e6',
 'f540b009-eb81-4a07-a351-dbe9b942f7e6',
 '12d6c278-a196-4df1-90b0-a9abf775b9e2',
 '5af3115d-4c17-4af8-9f33-3794a6cd649f',
 '19d2fb6e-9286-4204-8637-eeb85dcdbb01',
 'c8b1fb99-fdd5-436c-9d49-106a58cc4a3c',
 'ab84c526-bdc0-4fe6-bfbd-2d52fdb0de00',
 '62a0756c-044e-409a-ab22-dd6d1183689b',
 '6534f13f-9a2c-4e01-a595-a7de2df20ebd',
 '41f71205-7fe3-

#### Khởi tạo FAISS Vector Store

In [40]:
# Initialize the FAISS vector store
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.faiss import DistanceStrategy

index = faiss.IndexFlatL2(len(hf_embeddings.embed_query("hello world")))
vector_db_path = './vectorstore/db_faiss'

#### Lưu các chunks vector vào Faiss (Chỉ chạy khi chưa lưu dữ liệu)

In [41]:
vector_store = FAISS(
    embedding_function=hf_embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [42]:
vector_store.add_documents(documents=list_of_chunks, ids=uuids)

['d728b2ff-49d9-4035-a4e8-b788ffe707d6',
 'd2cdefd6-1adc-42c3-a366-9a908e3f2934',
 'e93aa4b1-f02a-4e46-9ba3-847d64d92d2f',
 'b31610d0-2749-47e8-b07a-24f3b58d4344',
 '1530f025-1610-47ef-9807-54ed5887736e',
 '9fb20865-fc8c-434b-889e-5a493f9b0f25',
 '618e2321-8405-44b8-8fc4-3242d0a046bd',
 '3701ae99-8c10-4913-aefc-ad91a660d2b6',
 '28da49cc-d318-405d-8203-1b922bd4dad4',
 '285c5355-24b1-4730-b605-1a5374f0cf59',
 '8be69d7f-cbf0-4cec-af66-90fa577eb1d9',
 '6a8a4a90-1830-451d-8755-99034f730d9a',
 'ee178c8b-0a33-4784-876b-80bbabeab522',
 '0ed09af2-c5ca-47c2-abe6-1769412a9c76',
 '370e3373-448c-476a-acea-eb0791fb2b4f',
 '9bc2e69f-5ccb-4156-855f-fdec568bc8e6',
 'f540b009-eb81-4a07-a351-dbe9b942f7e6',
 '12d6c278-a196-4df1-90b0-a9abf775b9e2',
 '5af3115d-4c17-4af8-9f33-3794a6cd649f',
 '19d2fb6e-9286-4204-8637-eeb85dcdbb01',
 'c8b1fb99-fdd5-436c-9d49-106a58cc4a3c',
 'ab84c526-bdc0-4fe6-bfbd-2d52fdb0de00',
 '62a0756c-044e-409a-ab22-dd6d1183689b',
 '6534f13f-9a2c-4e01-a595-a7de2df20ebd',
 '41f71205-7fe3-

In [43]:
# Lưu database vector
vector_store.save_local(vector_db_path)

#### Tải cơ sở vector Faiss (Chỉ chạy khi đã lưu dữ liệu)

In [26]:
# Tải lại FAISS index từ file
vector_store = FAISS.load_local(vector_db_path, hf_embeddings, allow_dangerous_deserialization = True)

In [27]:
print(vector_store.index_to_docstore_id)  # In ra mapping để kiểm tra

{0: '48447c2c-59a5-4a34-9d80-1d7b7efe9a83', 1: '0d17e3be-088a-455e-9180-8c6faa461e9e', 2: '2b1b6216-afe6-455c-b2c6-8167c98cb47a', 3: 'f1e7d338-fc14-4323-aba3-56e56cb9973a', 4: '151e6a54-a5b9-4bd3-95bc-eb831efe4d29', 5: 'b9ae763c-6368-43b8-bb6e-03d8c0b40ee4', 6: '8dce0278-d675-4ad7-8bf8-529d5f51d524', 7: '96106744-5b38-453b-86eb-aae759c00ba3', 8: '550c0f06-6144-48ce-8baf-e92175504a2f', 9: '1b965d5b-85c0-4d40-a4dd-690bc7aaa74c', 10: '341babfa-95e3-44a9-a09c-a11922525914', 11: 'cc3ed891-e702-41c7-b195-bbea44a4ee32', 12: '1b30aaf1-5f11-4c8f-ab4c-7276b369e2cb', 13: '7863e456-0ba7-441c-9036-98ec2053da79', 14: '9e959484-494f-4995-b87f-4b4fd435101a', 15: '1644d79d-142f-43ba-a145-ce5a822b3591', 16: '2ec2603f-05f2-4bd0-bd29-455ea3e43a6d', 17: 'c28613a6-bd13-4a91-97ac-e033e14187fd', 18: '0ca97436-16a0-413b-aaa7-f957cac09df5', 19: '5f3a3956-af99-451d-b2c2-a6e989f1a213', 20: 'c11b4965-9b92-49d1-9e8a-42f825f7c5b2', 21: '278a76b3-1bd6-4a81-86c9-5f5c99c6d43d', 22: 'fa45fca3-7526-499d-90e6-57905f291f7e

### III. Các kỹ thuật retriever

#### Kỹ thuật vector search của các semantic chunks

In [28]:
# Similarity search example with the vector store
semantic_results = vector_store.similarity_search(
    "Giải quyết các nút giao cắt với Xa lộ Hà Nội",
    k=10, 
)
for res in semantic_results:
    print(f"* {res.page_content} [{res.metadata}]")

* Mô tả chủ đề: Cải tạo và xây dựng các nút giao cắt giữa Xa lộ Hà Nội với các tuyến đường chính như Vành Đai 2, Vành Đai 3, Võ Văn Ngân – Lê Văn Việt nhằm đảm bảo lưu thông thông suốt. Nguyên nhân: Tình trạng ùn tắc tại các điểm giao cắt do lưu lượng xe cao và thiết kế chưa tối ưu. [{'parent': 150}]
* Mô tả chủ đề: Cải tạo giao lộ giữa Lê Văn Việt và Xa lộ Hà Nội để hỗ trợ giao thông kết nối hiệu quả với Song Hành. Nguyên nhân: Lưu lượng lớn phương tiện từ trung tâm TP.Thủ Đức đổ ra Xa lộ Hà Nội thường xuyên gây ùn tắc tại nút giao này. Phương pháp thực hiện: Xây dựng cầu vượt/hầm chui, mở rộng làn xe, phân làn rõ ràng và ứng dụng hệ thống điều khiển thông minh ITS. [{'parent': 245}]
* Cải tạo mặt đường, bổ sung làn riêng cho xe buýt, xe đạp. Phân kỳ xây dựng cầu vượt hoặc hầm chui tại các điểm giao cắt lớn. Ý nghĩa của hành vi:Giải quyết ùn tắc tuyến Song Hành là điểm mấu chốt để hỗ trợ sự phát triển bền vững của TP. Thủ Đức và thúc đẩy liên kết vùng trong khu vực Đông Sài Gòn. [{'pa

#### BM25 Retriever

In [29]:
# Sử dụng BM25 retriever từ LangChain
from langchain_community.retrievers import BM25Retriever

In [30]:
# bm25_params = {
#     "k1":1.25,
#     "b":0.5
# }

In [30]:
# Khởi tạo BM25 retriever với tham số tìm kiếm top 10 các kết quả liên quan nhất
bm25_retriever = BM25Retriever.from_documents(
  list_of_chunks, k = 10
)

In [31]:
bm25_results = bm25_retriever.invoke("Giải quyết các nút giao cắt với Xa lộ Hà Nội")
bm25_results

[Document(metadata={'parent': 150}, page_content='Mô tả chủ đề: Cải tạo và xây dựng các nút giao cắt giữa Xa lộ Hà Nội với các tuyến đường chính như Vành Đai 2, Vành Đai 3, Võ Văn Ngân – Lê Văn Việt nhằm đảm bảo lưu thông thông suốt. Nguyên nhân: Tình trạng ùn tắc tại các điểm giao cắt do lưu lượng xe cao và thiết kế chưa tối ưu.'),
 Document(metadata={'parent': 182}, page_content='Mô tả chủ đề: Tuyến đường Song Hành Xa lộ Hà Nội là tuyến đường phụ trợ quan trọng nằm bên cạnh Xa lộ Hà Nội, góp phần giảm tải lưu lượng giao thông cho tuyến chính và kết nối các khu vực lân cận. Nguyên nhân: Xa lộ Hà Nội là một trong những trục giao thông huyết mạch của TP.HCM với lưu lượng xe lớn, đặc biệt giờ cao điểm. Tuyến Song Hành được thiết kế để phân luồng phương tiện, giảm ùn tắc tại các nút giao thông lớn. Các tuyến đường giao cắt như Đỗ Xuân Hợp, Lê Văn Việt là các điểm then chốt trong mạng lưới giao thông kết nối quận 9, Thủ Đức với trung tâm TP. Phương pháp thực hiện: Thiết kế làn đường riêng 

#### Real - time semantic database data

In [32]:
def semantic_chunk_real_time(hf_embeddings, documents):
    tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-v2-m3")
    text_splitter = SemanticChunker(hf_embeddings,
                                breakpoint_threshold_type="percentile",
                                breakpoint_threshold_amount=85)
    # Split the documents and count the number of chunks in each document, as well as the number of tokens in each chunk
    count = 0
    total = 0
    list_of_chunks = []
    for idx,doc in enumerate(documents):
        chunks = text_splitter.create_documents([doc.page_content])
        print(f'Number of chunks: {len(chunks)} - Tokens of each chunk',end=' ')
        for chunk in chunks:
            text = chunk.page_content
            tokens = tokenizer.tokenize(text)
            num_tokens = len(tokens)
            if num_tokens > 1:
                total = total + 1
                # Use the parent document index as metadata to retrieve the parent document from the child chunk.
                chunk.metadata['parent'] = idx
                list_of_chunks.append(chunk)
            if num_tokens > 512:
                count = count + 1
            print(num_tokens, end =' ')
        print()
    print('Toltal chunks: ',total)
    print('Number of chunks which is larger than 512 tokens: ',count)
    return list_of_chunks

def filter_semantically_unique_chunks(chunks, vector_store, embedding_model, similarity_threshold=0.95):
    """
    Lọc các chunk chưa tồn tại trong FAISS dựa trên cosine similarity embedding.

    Args:
        chunks (List[Document]): Danh sách chunk cần kiểm tra.
        vector_store (FAISS): Vector store FAISS đang dùng.
        embedding_model: Mô hình HuggingFaceEmbeddings để tính embedding.
        similarity_threshold (float): Ngưỡng cosine similarity để xem là trùng lặp.

    Returns:
        List[Document]: Danh sách chunk không trùng lặp để thêm vào FAISS.
    """
    unique_chunks = []

    for chunk in chunks:
        embedding = embedding_model.embed_query(chunk.page_content)
        matches = vector_store.similarity_search_by_vector(embedding, k=1)

        if not matches:
            unique_chunks.append(chunk)
        else:
            existing_vector = embedding_model.embed_query(matches[0].page_content)
            cosine_sim = np.dot(embedding, existing_vector) / (
                np.linalg.norm(embedding) * np.linalg.norm(existing_vector)
            )
            if cosine_sim < similarity_threshold:
                unique_chunks.append(chunk)

    return unique_chunks

def add_chunks_to_faiss(chunks, vector_store, vector_db_path):
    """
    Thêm các chunk vào FAISS với UUID ngẫu nhiên và lưu lại.

    Args:
        chunks (List[Document]): Danh sách chunk đã lọc.
        vector_store (FAISS): Vector store FAISS đang dùng.
        vector_db_path (str): Đường dẫn thư mục lưu FAISS.

    Returns:
        int: Số lượng chunk đã thêm.
    """
    if not chunks:
        print("Không có chunk mới để thêm vào FAISS.")
        return 0

    uuids = [str(uuid4()) for _ in range(len(chunks))]
    vector_store.add_documents(chunks, ids=uuids)
    vector_store.save_local(vector_db_path)

    print(f"Đã thêm {len(chunks)} chunks mới vào FAISS.")
    return vector_store, len(chunks)

In [ ]:
# txt_chunks = semantic_chunk_real_time(hf_embeddings, txt_documents)
# unique_txt_chunks = filter_semantically_unique_chunks(txt_chunks, vector_store, hf_embeddings)
# vector_store, added_count = add_chunks_to_faiss(unique_txt_chunks, vector_store, vector_db_path)

Number of chunks: 1 - Tokens of each chunk 69 
Number of chunks: 1 - Tokens of each chunk 70 
Number of chunks: 1 - Tokens of each chunk 68 
Number of chunks: 1 - Tokens of each chunk 69 
Number of chunks: 1 - Tokens of each chunk 70 
Number of chunks: 1 - Tokens of each chunk 67 
Number of chunks: 1 - Tokens of each chunk 68 
Number of chunks: 1 - Tokens of each chunk 68 
Number of chunks: 1 - Tokens of each chunk 69 
Number of chunks: 1 - Tokens of each chunk 68 
Number of chunks: 1 - Tokens of each chunk 68 
Number of chunks: 1 - Tokens of each chunk 67 
Number of chunks: 1 - Tokens of each chunk 68 
Number of chunks: 1 - Tokens of each chunk 68 
Number of chunks: 1 - Tokens of each chunk 69 
Number of chunks: 1 - Tokens of each chunk 70 
Number of chunks: 1 - Tokens of each chunk 70 
Number of chunks: 1 - Tokens of each chunk 71 
Number of chunks: 1 - Tokens of each chunk 68 
Number of chunks: 1 - Tokens of each chunk 69 
Number of chunks: 1 - Tokens of each chunk 69 
Number of chu

In [33]:
# Similarity search example with the vector store
semantic_results = vector_store.similarity_search(
    "Ngày 6 tháng 6 tại camera đường Xa lộ Hà Nội - Lê Văn Việt có bao nhiêu lượt xe với vận tốc trung bình bao nhiêu ?",
    k = 10, 
)
for res in semantic_results:
    print(f"* {res.page_content} [{res.metadata}]")

* Mô tả chủ đề: Khuyến khích sử dụng các thiết bị hỗ trợ lái xe an toàn như camera hành trình, cảm biến cảnh báo va chạm để nâng cao khả năng nhận diện tình huống. Nguyên nhân: Người lái xe thiếu thông tin về môi trường xung quanh, dẫn đến phản ứng chậm với các tình huống nguy hiểm. [{'parent': 357}]
* Xây dựng hệ thống cảnh báo thông minh hỗ trợ lái xe như camera giám sát, gờ giảm tốc, đèn báo nguy hiểm. Đào tạo bắt buộc về lái xe an toàn tại các trung tâm sát hạch và điểm đăng ký giấy phép lái xe. Kết hợp giữa chính quyền địa phương và lực lượng CSGT để tăng kiểm tra, xử phạt nghiêm các hành vi vi phạm. Ý nghĩa của hành vi:Giảm thiểu tai nạn giao thông và thương vong tại các khu vực giao thông phức tạp. Góp phần xây dựng văn hóa giao thông văn minh, an toàn và thân thiện. Nâng cao chất lượng cuộc sống đô thị, đặc biệt ở khu vực phía Đông Thành phố Hồ Chí Minh. Tạo điều kiện lưu thông thông suốt, nâng cao hiệu quả khai thác của tuyến Song Hành Xa lộ Hà Nội sau khi được đầu tư mở rộng.

In [34]:
def build_realtime_faiss_index(hf_embeddings, txt_chunks):
    uuids = [str(uuid4()) for _ in range(len(txt_chunks))]
    index = faiss.IndexFlatL2(len(hf_embeddings.embed_query("hello world")))

    vector_store = FAISS(
        embedding_function=hf_embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
    vector_store.add_documents(documents=txt_chunks, ids=uuids)
    
    return vector_store, txt_chunks

In [35]:
txt_chunks = semantic_chunk_real_time(hf_embeddings, txt_documents)
realtime_faiss, txt_chunks = build_realtime_faiss_index(hf_embeddings, txt_chunks)

Number of chunks: 1 - Tokens of each chunk 69 
Number of chunks: 1 - Tokens of each chunk 67 
Number of chunks: 1 - Tokens of each chunk 68 
Number of chunks: 1 - Tokens of each chunk 69 
Number of chunks: 1 - Tokens of each chunk 67 
Number of chunks: 1 - Tokens of each chunk 68 
Number of chunks: 1 - Tokens of each chunk 66 
Number of chunks: 1 - Tokens of each chunk 67 
Number of chunks: 1 - Tokens of each chunk 68 
Number of chunks: 1 - Tokens of each chunk 67 
Number of chunks: 1 - Tokens of each chunk 67 
Number of chunks: 1 - Tokens of each chunk 68 
Number of chunks: 1 - Tokens of each chunk 66 
Number of chunks: 1 - Tokens of each chunk 67 
Number of chunks: 1 - Tokens of each chunk 66 
Number of chunks: 1 - Tokens of each chunk 67 
Number of chunks: 1 - Tokens of each chunk 69 
Number of chunks: 1 - Tokens of each chunk 69 
Number of chunks: 1 - Tokens of each chunk 69 
Number of chunks: 1 - Tokens of each chunk 69 
Number of chunks: 1 - Tokens of each chunk 70 
Number of chu

#### Real - time bm25

In [36]:
def BM25_Retriever(list_of_chunks):
    # Khởi tạo BM25 retriever với tham số tìm kiếm top 10 các kết quả liên quan nhất
    bm25_retriever = BM25Retriever.from_documents(
        list_of_chunks, k = 10
    )
    return bm25_retriever

In [ ]:
# Khởi tạo BM25 retriever với tham số tìm kiếm top 10 các kết quả liên quan nhất
# list_of_chunks = list_of_chunks + txt_chunks
# bm25_retriever = BM25_Retriever(list_of_chunks)

In [38]:
bm25_results = bm25_retriever.invoke("Ngày 6 tháng 6 tại camera đường Xa lộ Hà Nội - Lê Văn Việt có bao nhiêu lượt xe với vận tốc trung bình bao nhiêu ?")
bm25_results

[Document(metadata={'parent': 190}, page_content='Vào ngày 6 tháng 6 năm 2025, tại camera đặt tại Xa lộ Hà Nội – Đường Lê Văn Việt, khu vực Phường Hiệp Phú của TP Thủ Đức, TP.HCM, đã ghi nhận tổng cộng 71 lượt xe. Tốc độ trung bình là 22.98 km/h.'),
 Document(metadata={'parent': 187}, page_content='Vào ngày 10 tháng 6 năm 2025, tại camera đặt tại Xa lộ Hà Nội – Đường Lê Văn Việt, khu vực Phường Hiệp Phú của TP Thủ Đức, TP.HCM, đã ghi nhận tổng cộng 471 lượt xe. Tốc độ trung bình là 26.41 km/h.'),
 Document(metadata={'parent': 172}, page_content='Vào ngày 6 tháng 6 năm 2025, trong khung giờ 17:00–17:30, tốc độ trung bình ghi nhận tại camera đặt tại Xa lộ Hà Nội – Đường Lê Văn Việt, khu vực Phường Hiệp Phú – TP Thủ Đức, TP.HCM là 22.98 km/h.'),
 Document(metadata={'parent': 168}, page_content='Vào ngày 10 tháng 6 năm 2025, trong khung giờ 20:00–20:30, tốc độ trung bình ghi nhận tại camera đặt tại Xa lộ Hà Nội – Đường Lê Văn Việt, khu vực Phường Hiệp Phú – TP Thủ Đức, TP.HCM là 26.41 km/h

#### BGE-m3 Reranker

In [39]:
# Lấy kết quả chunks từ BM25 Retriever và FAISS vector search
content = set()
retrieval_docs = []

for result in semantic_results:
  if result.page_content not in content:
    content.add(result.page_content)
    retrieval_docs.append(result)

for result in bm25_results:
  if result.page_content not in content:
    content.add(result.page_content)
    retrieval_docs.append(result)

len(retrieval_docs)

17

In [40]:
# Định nghĩa max_length cho padding
max_length = 512  # Hoặc bạn có thể thay đổi giá trị này tùy vào yêu cầu

# Tokenize the text with padding and truncation (Mã hóa dữ liệu)
encoded_inputs = tokenizer(
    "Ngày 6 tháng 6 tại camera đường Xa lộ Hà Nội - Lê Văn Việt có bao nhiêu lượt xe với vận tốc trung bình bao nhiêu ?",  # văn bản muốn token hóa
    padding='max_length',    # Đảm bảo tất cả các văn bản đều có cùng độ dài
    truncation=False,         # Cắt văn bản nếu quá dài
    max_length=max_length,   # Chiều dài tối đa
    return_tensors="pt"      # Trả về kết quả dưới dạng tensors (nếu cần)
)

In [41]:
# Sử dụng mô hình BAAI/bge-reranker-v2-m3 để xếp hạng các kết quả chunks dựa trên số điểm tính toán
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) # Thiết lập use_fp16 = True để tăng tốc tính toán với một sự giảm hiệu suất nhẹ.
pairs = [["Ngày 6 tháng 6 tại camera đường Xa lộ Hà Nội - Lê Văn Việt có bao nhiêu lượt xe với vận tốc trung bình bao nhiêu ?",doc.page_content] for doc in retrieval_docs]
score = reranker.compute_score(pairs,normalize = True)
score

pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 253.16it/s]
c:\Users\ADMIN\OneDrive\Documents\Btap_Code\VisualStudioCode\Python\TLCN_HealthChatbot\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


[0.0003935755935111022,
 0.012920473039078978,
 0.0025311113093281793,
 0.0035657366623610266,
 0.0009436192455752618,
 0.0019723357407638166,
 0.014172366191301682,
 0.9999151889582765,
 0.998224739467894,
 0.9976768374773688,
 0.975154563105731,
 0.9753431444580398,
 0.9788748081040518,
 0.9872281325067137,
 0.9946567440438505,
 0.9925969859111274,
 0.993255010457908]

#### Smart Retriever

In [42]:
# Xây dựng, tổng hợp các kỹ thuật truy vấn nâng cao thành lớp 'Retriever'
class Retriever:
  def __init__(self, semantic_retriever, bm25_retriever, reranker, documents):
    self.semantic_retriever = semantic_retriever
    self.bm25_retriever = bm25_retriever
    self.reranker = reranker
    self.documents = documents

  def __call__(self,query):
    semantic_results = self.semantic_retriever.similarity_search(
      query,
      k=10,
    )
    bm25_results = self.bm25_retriever.invoke(query)

    content = set()
    retrieval_docs = []

    for result in semantic_results:
      if result.page_content not in content:
        content.add(result.page_content)
        retrieval_docs.append(result)

    for result in bm25_results:
      if result.page_content not in content:
        content.add(result.page_content)
        retrieval_docs.append(result)

    pairs = [[query, doc.page_content] for doc in retrieval_docs]

    scores = self.reranker.compute_score(pairs, normalize = True)

    # Lấy tài liệu nguồn từ phần tử con dựa trên điểm số ngưỡng
    context_1 = []
    context_2 = []
    context = []
    parent_ids = set()
    for i in range(len(retrieval_docs)):
      # Điểm liên quan >= 0.65 sẽ được sử dụng làm kiểu ngữ cảnh 1 (chỉ ra sự liên quan cao hơn đối với truy vấn).
      if scores[i] >= 0.65:
        parent_idx = retrieval_docs[i].metadata['parent']
        if parent_idx not in parent_ids:
          parent_ids.add(parent_idx)
          context_1.append(self.documents[parent_idx])
      # Điểm liên quan >= 0.1 sẽ được sử dụng làm kiểu ngữ cảnh 2 (chỉ ra sự liên quan trung bình đến thấp đối với truy vấn).
      elif scores[i] >= 0.1:
        parent_idx = retrieval_docs[i].metadata['parent']
        if parent_idx not in parent_ids:
          parent_ids.add(parent_idx)
          context_2.append(self.documents[parent_idx])
      
    if len(context_1) > 0:
      print('Context 1')
      context=context_1
      index = 1
    elif len(context_2) > 0:
      print('Context 2')
      context=context_2
      index = 2
    else:
      # Nếu điểm liên quan < 0.1, điều này chỉ ra rằng không có tài liệu liên quan.
      print('No relevant context')
    return context, index

In [43]:
# Kiểm tra bộ truy xuất Smart Retriever
retriever = Retriever(semantic_retriever = vector_store, bm25_retriever = bm25_retriever, reranker = reranker, documents = documents)
context, _ = retriever("Các giải pháp xử lý tình trạng kẹt xe ở đường Xa lộ Hà Nội giao với Lê Văn Việt")
context

pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 384.20it/s]
c:\Users\ADMIN\OneDrive\Documents\Btap_Code\VisualStudioCode\Python\TLCN_HealthChatbot\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Context 1


[Document(metadata={'title': 'Tiêu đề: Cải tạo nút giao Xa lộ Hà Nội - Lê Văn Việt', 'category': 'Phân bố luồng giao thông và Quy hoạch đô thị'}, page_content='Mô tả chủ đề: Cải tạo giao lộ giữa Lê Văn Việt và Xa lộ Hà Nội để hỗ trợ giao thông kết nối hiệu quả với Song Hành.\nNguyên nhân: Lưu lượng lớn phương tiện từ trung tâm TP.Thủ Đức đổ ra Xa lộ Hà Nội thường xuyên gây ùn tắc tại nút giao này.\nPhương pháp thực hiện: Xây dựng cầu vượt/hầm chui, mở rộng làn xe, phân làn rõ ràng và ứng dụng hệ thống điều khiển thông minh ITS.\nÝ nghĩa của hành vi:Tối ưu hóa hướng luồng giao thông từ Lê Văn Việt sang Song Hành, giảm xung đột trực tiếp trên Xa lộ Hà Nội.'),
 Document(metadata={'title': 'Tiêu đề: Giải quyết các nút giao cắt với Xa lộ Hà Nội', 'category': 'Phân bố luồng giao thông và Quy hoạch đô thị'}, page_content='Mô tả chủ đề: Cải tạo và xây dựng các nút giao cắt giữa Xa lộ Hà Nội với các tuyến đường chính như Vành Đai 2, Vành Đai 3, Võ Văn Ngân – Lê Văn Việt nhằm đảm bảo lưu thông t

- Retriever real time

In [ ]:
class Retriever2:
    def __init__(self, semantic_retriever, bm25_retriever, semantic_retriever_realtime, bm25_real_time, reranker, documents):
        self.semantic_retriever = semantic_retriever
        self.bm25_retriever = bm25_retriever
        self.semantic_retriever_realtime = semantic_retriever_realtime
        self.bm25_real_time = bm25_real_time
        self.reranker = reranker
        self.documents = documents

    def __call__(self, query):
        semantic_results = self.semantic_retriever.similarity_search(query, k=10)
        bm25_results = self.bm25_retriever.invoke(query)

        realtime_results = self.semantic_retriever_realtime.similarity_search(query, k=10)
        bm25_online_results = self.bm25_real_time.invoke(query)

        content = set()
        retrieval_docs = []

        for result in realtime_results + bm25_online_results + semantic_results + bm25_results:
            if result.page_content not in content:
                content.add(result.page_content)
                retrieval_docs.append(result)

        pairs = [[query, doc.page_content] for doc in retrieval_docs]

        scores = self.reranker.compute_score(pairs, normalize = True)

        # Lấy tài liệu nguồn từ phần tử con dựa trên điểm số ngưỡng
        context_1 = []
        context_2 = []
        context_3 = []
        context = []
        parent_ids = set()
        for i in range(len(retrieval_docs)):
            # Điểm liên quan >= 0.6 sẽ được sử dụng làm kiểu ngữ cảnh 1 (chỉ ra sự liên quan cao hơn đối với truy vấn).
            if scores[i] >= 0.8:
                parent_idx = retrieval_docs[i].metadata['parent']
                if parent_idx not in parent_ids:
                    parent_ids.add(parent_idx)
                    context_1.append(self.documents[parent_idx])

            elif scores[i] >= 0.6:
                parent_idx = retrieval_docs[i].metadata['parent']
                if parent_idx not in parent_ids:
                    parent_ids.add(parent_idx)
                    context_2.append(self.documents[parent_idx])

            # Điểm liên quan >= 0.2 sẽ được sử dụng làm kiểu ngữ cảnh 2 (chỉ ra sự liên quan trung bình đến thấp đối với truy vấn).
            elif scores[i] >= 0.2:
                parent_idx = retrieval_docs[i].metadata['parent']
                if parent_idx not in parent_ids:
                    parent_ids.add(parent_idx)
                    context_3.append(self.documents[parent_idx])
        
        if len(context_1) > 0:
            print('Context 1')
            context=context_1
        elif len(context_2) > 0:
            print('Context 2')
            context=context_2
        elif len(context_3) > 0:
            print('Context 3')
            context=context_3
        else:
            # Nếu điểm liên quan < 0.1, điều này chỉ ra rằng không có tài liệu liên quan.
            print('No relevant context')
        return context


### IV. Tải mô hình và xây dựng câu trả lời

#### Tải thư viện

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms.ctransformers import CTransformers
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import BitsAndBytesConfig

#### Tải mô hình đã fine-tuned 

In [4]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
load_in_4bit = True
model = AutoPeftModelForCausalLM.from_pretrained(
    "./model/HCM_4BIT00007", # YOUR MODEL YOU USED FOR TRAINING
    load_in_4bit = load_in_4bit,
)
tokenizer = AutoTokenizer.from_pretrained("./model/HCM_4BIT00007")

TypeError: LoraConfig.__init__() got an unexpected keyword argument 'exclude_modules'

In [12]:
# Đường dẫn tới mô hình và tokenizer
# model_path = r"C:\Users\ADMIN\OneDrive\Documents\Btap_Code\VisualStudioCode\Python\TLCN_HealthChatbot\mlperf_env_P2\mlperf_env\model\4BIT00006"
model_path = "./model/4BIT00006"

# Tham số lượng tử hóa mô hình
use_4bit, bnb_4bit_compute_dtype, bnb_4bit_quant_type, use_nested_quant = True, "float16", "nf4", False 
device_map = {"": 0}
# Tải tokenizer và mô hình với tham số QLoRA 
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

tokenizer = AutoTokenizer.from_pretrained(model_path)
# Thiết lập pad token thành eos_token (có 2 trường hợp)
    ## Lựa chọn 1: Thiết lập eos_token thành pad_token
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token 

    # Hoặc, lựa chọn 2: Thêm một padding token mới
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = AutoModelForCausalLM.from_pretrained(
        model_path,
        quantization_config=bnb_config,
        device_map=device_map
    )

# model.config.use_cache = False
# model.config.pretraining_tp = 1

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

Exception: data did not match any variant of untagged enum ModelWrapper at line 1251003 column 3

#### Tạo Prompt Template và QA Question

In [40]:
system_prompt = f"""TrafficChat ViVi – Chuyên gia tư vấn quy hoạch đô thị thông minh
Vai trò:
ViVi hỗ trợ nhà quy hoạch và kỹ sư giao thông phân tích dữ liệu vận tốc, lưu lượng xe và đề xuất các giải pháp tối ưu hạ tầng và chính sách giao thông.

Chức năng chính:
Phân tích dữ liệu: vận tốc, lưu lượng, mật độ, điểm ùn tắc và tai nạn.
Đề xuất quy hoạch: thiết kế đường phố, biện pháp giảm tốc, hạ tầng cho người đi bộ, ITS.
Hỗ trợ chính sách: đánh giá và điều chỉnh chính sách giao thông, cập nhật xu hướng công nghệ.

Nguyên tắc:
Chính xác và minh bạch
Ưu tiên an toàn, bền vững
Tùy chỉnh theo ngữ cảnh từng khu vực
Lưu ý: Nếu không biết thì bảo là không biết, không được trả lời bịa đặt!!
"""

In [59]:
# Hàm tạo câu trả lời từ mô hình Llama
def generate_answer_from_documents(query, context, model, tokenizer):
    formatted_prompt = f"""
        Bạn là một trợ lý giao thông thông minh. Dựa vào các thông tin đã được thu thập, hãy trả lời chính xác câu hỏi của người dùng.
        # Câu hỏi:
        {query}

        # Dữ liệu liên quan:
        {context}
    """
    
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": formatted_prompt}]
    # Mã hóa prompt 
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        # padding='max_length',  # Padding đến độ dài tối đa
        truncation=True,       # Cắt ngắn nếu cần thiết
        return_tensors="pt"
    ).to(model.device)
    
    attention_mask = (input_ids != tokenizer.pad_token_id).long()

    # Tạo câu trả lời từ mô hình
    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,  # Sử dụng attention mask
        max_new_tokens=512,
        min_length=30, 
        do_sample=True, # Quyết định có sử dụng kỹ thuật sampling ngẫu nhiên hay không (True là có, False là không).
        temperature=0.7,    # Điều chỉnh mức độ ngẫu nhiên trong quá trình sinh câu trả lời (giá trị thấp hơn -> ít ngẫu nhiên).
        top_p=0.9,  # Giới hạn tỷ lệ xác suất của các token sẽ được chọn
        min_p=0.1,
        eos_token_id=terminators
    )

    # Giải mã và trả về câu trả lời
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

# Kiểm tra với Retriever 
def get_answer_with_retriever(query, retriever, model, tokenizer):
    # Sử dụng retriever để lấy context (dữ liệu từ các tài liệu liên quan)
    context, index = retriever(query)
    print(f"Index: {index}")
    # Nếu context có dữ liệu, sử dụng mô hình để tạo câu trả lời
    if context:
        context_str = "\n".join([doc.page_content for doc in context])  # Chuyển tất cả các tài liệu thành chuỗi
        return generate_answer_from_documents(query, context_str, model, tokenizer)
    else:
        return "Không tìm thấy thông tin liên quan để trả lời câu hỏi."

In [65]:
# Kiểm tra với Retriever 
def get_answer_with_two_retrievers(query, realtime_retriever, static_retriever, model, tokenizer, reranker=None, top_k=5):
    # Sử dụng retriever để lấy context (dữ liệu từ các tài liệu liên quan)
    realtime_docs, index_real_time = realtime_retriever(query)
    static_docs, index_static = static_retriever(query)
    
    if int(index_real_time) < int(index_static):
        all_docs = realtime_docs
    elif int(index_static) < int(index_real_time):
        all_docs = static_docs
    else:
        # Gộp kết quả, nhưng đặt realtime lên trước
        all_docs = realtime_docs + static_docs

        # Rerank nếu có
        if reranker:
            pairs = [[query, doc.page_content] for doc in all_docs]
            scores = reranker.compute_score(pairs, normalize=True)
            # Gắn điểm vào tài liệu để sắp xếp
            for i in range(len(all_docs)):
                all_docs[i].metadata['score'] = scores[i]
            # Sắp xếp giảm dần theo điểm
            all_docs = sorted(all_docs, key=lambda x: x.metadata['score'], reverse=True)

        # Cắt lấy top_k kết quả sau khi rerank (hoặc đơn giản là ưu tiên real-time)
        selected_docs = all_docs[:top_k] if len(all_docs) >= top_k else all_docs
        all_docs = selected_docs

    # Nếu có dữ liệu thì sinh câu trả lời
    if all_docs:
        context_str = "\n".join([doc.page_content for doc in all_docs])
        return generate_answer_from_documents(query, context_str, model, tokenizer)
    else:
        return "Không tìm thấy thông tin liên quan để trả lời câu hỏi."

#### Thử nghiệm thực tế câu hỏi

In [61]:
query = "Ngày 6 tháng 6 tại camera đường Xa lộ Hà Nội giao nhau với Lê Văn Việt có tổng hợp tất cả bao nhiêu lượt xe với vận tốc trung bình bao nhiêu ?"
retriever = Retriever(semantic_retriever = vector_store, bm25_retriever = bm25_retriever, reranker = reranker, documents = documents)

# Gọi hàm để lấy câu trả lời từ retriever và mô hình Llama
answer = get_answer_with_retriever(query, retriever, model, tokenizer)
print("Answer:", answer)

pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 500.45it/s]
c:\Users\ADMIN\OneDrive\Documents\Btap_Code\VisualStudioCode\Python\TLCN_HealthChatbot\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Context 2
Index: 2
Answer: Dựa vào thông tin đã được thu thập, vào ngày 6 tháng 6, camera đường Xa lộ Hà Nội giao nhau với Lê Văn Việt ghi nhận tổng cộng 1000 lượt xe với vận tốc trung bình là 60 km/h.


In [62]:
query = "Ngày 6 tháng 6 tại camera đường Xa lộ Hà Nội giao nhau với Lê Văn Việt có bao nhiêu lượt xe với vận tốc trung bình bao nhiêu ?"
realtime_bm25 = BM25_Retriever(txt_chunks)
retriever = Retriever(semantic_retriever = realtime_faiss, bm25_retriever = realtime_bm25, reranker = reranker, documents = txt_documents)

# Gọi hàm để lấy câu trả lời từ retriever và mô hình Llama
answer = get_answer_with_retriever(query, retriever, model, tokenizer)
print("Answer:", answer)

pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 500.33it/s]
c:\Users\ADMIN\OneDrive\Documents\Btap_Code\VisualStudioCode\Python\TLCN_HealthChatbot\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Context 1
Index: 1
Answer: Dựa vào dữ liệu đã cung cấp, vào ngày 6 tháng 6 năm 2025, tại camera đường Xa lộ Hà Nội giao nhau với Lê Văn Việt, có tổng cộng 71 lượt xe. Tốc độ trung bình ghi nhận tại camera này là 22.98 km/h.


In [66]:
query = "Ngày 6 tháng 6 tại camera đường Xa lộ Hà Nội giao nhau với Lê Văn Việt có bao nhiêu lượt xe với vận tốc trung bình bao nhiêu ?"
realtime_bm25 = BM25_Retriever(txt_chunks)
realtime_retriever = Retriever(semantic_retriever = realtime_faiss, bm25_retriever = realtime_bm25, reranker = reranker, documents = txt_documents)
static_retriever = Retriever(semantic_retriever = vector_store, bm25_retriever = bm25_retriever, reranker = reranker, documents = documents)

# Gọi hàm để lấy câu trả lời từ retriever và mô hình Llama
answer = get_answer_with_two_retrievers(query, realtime_retriever, static_retriever, model, tokenizer, reranker, 10)
print("Answer:", answer)

pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 500.87it/s]
c:\Users\ADMIN\OneDrive\Documents\Btap_Code\VisualStudioCode\Python\TLCN_HealthChatbot\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Context 1


pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 450.32it/s]
c:\Users\ADMIN\OneDrive\Documents\Btap_Code\VisualStudioCode\Python\TLCN_HealthChatbot\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Context 2
Answer: Vào ngày 6 tháng 6 năm 2025, tại camera đường Xa lộ Hà Nội giao nhau với Lê Văn Việt, có tổng cộng 71 lượt xe được ghi nhận. Tốc độ trung bình của các phương tiện lưu thông trong khoảng thời gian này là 22.98 km/h. Hướng lưu thông đông nhất là hướng Bắc.


In [ ]:
# Kiểm tra bộ truy xuất Smart Retriever 2
query = "Ngày 6 tháng 6 tại camera đường Xa lộ Hà Nội - Lê Văn Việt có bao nhiêu lượt xe với vận tốc trung bình bao nhiêu ?"
retriever = Retriever2(semantic_retriever = vector_store, 
                      bm25_retriever = bm25_retriever,
                      semantic_retriever_realtime = realtime_faiss,
                      reranker = reranker,
                      documents = (documents))

# Gọi hàm để lấy câu trả lời từ retriever và mô hình Llama
answer = get_answer_with_retriever(query, retriever, model, tokenizer)
print("Answer:", answer)

pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 201.92it/s]
c:\Users\ADMIN\OneDrive\Documents\Btap_Code\VisualStudioCode\Python\TLCN_HealthChatbot\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Context 1
Answer: Xin lỗi, như một mô hình ngôn ngữ, ViVi không thể truy cập vào dữ liệu cụ thể như camera đường Xa lộ Hà Nội - Lê Văn Việt. Tuy nhiên, ViVi có thể giải thích về lý thuyết dòng xe và vai trò của nó trong nghiên cứu và thực tiễn giao thông.

Lý thuyết dòng xe là một phần của lý thuyết cơ học, nghiên cứu về các hiện tượng liên quan đến chuyển động của các đối tượng trong hệ thống giao thông. Nó giúp hiểu về các yếu tố như lưu lượng, mật độ và tốc độ của các phương tiện di chuyển, và cách chúng tương tác với nhau và với môi trường xung quanh.

Mô hình dòng xe thường được xây dựng bằng cách phân tích dữ liệu từ các nguồn như dữ liệu từ camera, dữ liệu từ cảm biến, và dữ liệu từ các phương tiện giao thông khác. Mô hình này cung cấp một cách để hiểu về các mẫu di chuyển của các phương tiện, xác định điểm nghẽn, ước lượng lưu lượng cực đại và xác định điều kiện hoạt động tối ưu.

Nghiên cứu về lý thuyết dòng xe đã dẫn đến việc phát triển các mô hình điều phối, giúp đánh giá hi